In [1]:
import re
import glob
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

In [2]:
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.utils import shuffle

## Train same model as previous notebook

In [3]:
# Load simulated data: load complete training and validation sets.
np.random.seed(11)

# Training set should be one concatenated shuffled list of simulated examples
# Validation set similarly, but also keep information about simulation parameters
# to evaluate performance in specific simulation conditions.
bounds_files = glob.glob("../data/simulData/*_bmBounds.tab")
intensities_files = [re.sub(r"_bmBounds.tab$", "_intensities.tab", bounds_file)
                     for bounds_file in bounds_files]
y, x = shuffle(
    np.concatenate([np.loadtxt(bounds_file) for bounds_file in bounds_files]),
    np.concatenate([np.loadtxt(intensities_file) for intensities_file in intensities_files])
)

In [4]:
def overlap_metric(y_true, y_predicted):
    y_pred_round = tf.round(y_predicted)
    max_start = tf.math.maximum(y_true[:, 0], y_pred_round[:, 0])
    min_end = tf.math.minimum(y_true[:, 1], y_pred_round[:,1])
    overlap = tf.math.maximum(0.0, min_end - max_start)
    longest_bounds = tf.math.maximum(
        y_true[:, 1] + 1 - y_true[:, 0], 
        y_pred_round[:, 1] + 1 - y_pred_round[:, 0]
        )
    return overlap / longest_bounds

In [5]:
model = tf.keras.Sequential([
        layers.Dense(40, activation='relu'),
        layers.Dense(2, activation='linear')
    ])
model.compile(
    optimizer=tf.train.AdamOptimizer(learning_rate=0.01),
    loss='mse',
    metrics=['mae', overlap_metric]
    )

In [6]:
history = model.fit(x, y, epochs=1000, batch_size=8192, validation_split=0.33)

Train on 60299 samples, validate on 29701 samples
Epoch 1/1000
60299/60299 [==============================] - 1s 16us/step - loss: 2724.2504 - mean_absolute_error: 42.3468 - overlap_metric: 0.0151 - val_loss: 668.3637 - val_mean_absolute_error: 20.7189 - val_overlap_metric: 0.0368
Epoch 2/1000
60299/60299 [==============================] - 1s 10us/step - loss: 603.0986 - mean_absolute_error: 20.0796 - overlap_metric: 0.0400 - val_loss: 338.8343 - val_mean_absolute_error: 14.6950 - val_overlap_metric: 0.0467
Epoch 3/1000
60299/60299 [==============================] - 1s 10us/step - loss: 327.1732 - mean_absolute_error: 14.4480 - overlap_metric: 0.0600 - val_loss: 303.0478 - val_mean_absolute_error: 14.3558 - val_overlap_metric: 0.0481
Epoch 4/1000
60299/60299 [==============================] - 1s 11us/step - loss: 194.0972 - mean_absolute_error: 11.1292 - overlap_metric: 0.0786 - val_loss: 155.1968 - val_mean_absolute_error: 10.0504 - val_overlap_metric: 0.1071
Epoch 5/1000
60299/60299 

Epoch 37/1000
60299/60299 [==============================] - 1s 11us/step - loss: 10.7712 - mean_absolute_error: 2.5158 - overlap_metric: 0.3994 - val_loss: 10.8220 - val_mean_absolute_error: 2.5122 - val_overlap_metric: 0.4029
Epoch 38/1000
60299/60299 [==============================] - 1s 10us/step - loss: 10.5516 - mean_absolute_error: 2.4892 - overlap_metric: 0.4029 - val_loss: 10.6028 - val_mean_absolute_error: 2.4839 - val_overlap_metric: 0.4008
Epoch 39/1000
60299/60299 [==============================] - 1s 10us/step - loss: 10.3424 - mean_absolute_error: 2.4630 - overlap_metric: 0.4073 - val_loss: 10.4336 - val_mean_absolute_error: 2.4649 - val_overlap_metric: 0.4123
Epoch 40/1000
60299/60299 [==============================] - 1s 11us/step - loss: 10.1518 - mean_absolute_error: 2.4391 - overlap_metric: 0.4114 - val_loss: 10.2688 - val_mean_absolute_error: 2.4436 - val_overlap_metric: 0.4103
Epoch 41/1000
60299/60299 [==============================] - 1s 10us/step - loss: 9.9826

KeyboardInterrupt: 

Check same metrics as previous notebook.

In [ ]:
print("Mean absolute error", history.history['val_mean_absolute_error'][-1])
print("Overlap metric", history.history['val_overlap_metric'][-1])

## Validation on different datasets

Simulated datasets, separately

In [ ]:
bounds_files = glob.glob("../data/simulData/*_bmBounds.tab")
intensities_files = [re.sub(r"_bmBounds.tab$", "_intensities.tab", bounds_file)
                     for bounds_file in bounds_files]


In [ ]:
adip_metrics = defaultdict(list)
scale_metrics = defaultdict(list)
for bounds_file, intensities_file in zip(bounds_files, intensities_files):
    test_y, test_x = shuffle(np.loadtxt(bounds_file), np.loadtxt(intensities_file))
    values = model.evaluate(test_x, test_y)
    m = re.search(r"/scale(.*)-adip(.*)_bmBounds.tab$", bounds_file)
    scale, adip = [float(x) for x in m.groups()]
    overlap_index = model.metrics_names.index("overlap_metric")
    overlap = values[overlap_index]
    adip_metrics[adip].append(overlap)
    scale_metrics[scale].append(overlap)

In [ ]:
adiposities = sorted(adip_metrics)
adip_means = [np.mean(adip_metrics[adip]) for adip in adiposities]

In [ ]:
adip_means

In [ ]:
plt.plot(adiposities, adip_means)
plt.show()

In [ ]:
pts

data

In [ ]:
?plt.plot

## Real data predictions

In [ ]:
real_data_file = "../data/realData/FS_1334172_layers.csv"
raw = np.loadtxt(real_data_file, delimiter=";")
real_data = np.transpose(raw)
preds = model.predict(real_data)

In [ ]:
np.savetxt("predictions.txt", preds, fmt="%d")